In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import rushd as rd
import yaml
import scipy as sp
import numpy as np

from pathlib import Path
from typing import Any, Dict, List, Set, Tuple, Union, Optional

datadir = Path(Path('../datadir.txt').read_text())

basepath = rd.datadir/'instruments'/'data'/'qPCR'/'emma'/'command'/'2024.10.15_command_primer_validation'
output_path = rd.rootdir/'output'/'qPCR'/'primer_validation'

In [ ]:
rd.plot.plot_well_metadata(basepath/'metadata.yaml')

In [ ]:
with (basepath/'metadata.yaml').open() as f:
    metadata_dict = yaml.safe_load(f)['metadata']
    metadata = pd.DataFrame({
        k: rd.well_mapper.well_mapping(v) for k,v in metadata_dict.items()
    })

In [ ]:
qpcr_cp = pd.read_csv(basepath/'2024.10.16_ELP_qPCR_command_Cp.txt', sep='\t', header=1)
df = qpcr_cp.join(metadata, on='Pos')
df = df.loc[df['Cp']<35]

In [ ]:
g = sns.scatterplot(data=df, x='condition', y='Cp', hue='primer_set')
g.set_xscale('log')
plt.savefig(str(output_path)+'/qPCR_Ct.svg',bbox_inches='tight')

In [ ]:
df['log2_template'] = df['condition'].apply(np.log2)
df['log10_template'] = df['condition'].apply(np.log10)
display(df)

In [ ]:
primers = ['mRuby2-1', 'mRuby2-2', 'FXN', 'EGFP-1', 'EGFP-2', 'FMRP-1', 'FMRP-2']
efficiencies = ['83.04%', '99.50%', '90.39%', '70.98%', '73.86%', '59.21%', '59.74%']

for i in range(7):
    plt.clf()
    plot_df = df.loc[(df['primer_set']==primers[i])&(df['condition']>0)]
    g = sns.regplot(data=plot_df, x='log10_template', y='Cp')
    g.set_xlabel('log(DNA template, pg)')
    g.set_ylabel('Cp')
    g.set_title(primers[i])
    g.set_ylim(3,33)
    slope, intercept, r, p, sterr = sp.stats.linregress(x=g.get_lines()[0].get_xdata(), y=g.get_lines()[0].get_ydata())
    plt.text(2, 31, 'y = ' + str(round(intercept,3)) + ' + ' + str(round(slope,3)) + 'x')
    plt.text(2, 29, 'efficiency = ' + efficiencies[i])
    plt.savefig(str(output_path)+'/'+primers[i]+'_regression.svg',bbox_inches='tight')
    plt.show()